№1 (1 балл)
Имеется синхронная версия программы для подсчета значений факториала чисел. Используя модуль asyncio сделайте программу асинхронной. В этом случае в каком порядке завершаются задачи? С->B->A Из-за количества необходимых циклов для вычисления факториалов.

In [16]:
import asyncio

async def factorial(name, number): 
    f = 1 
    for i in range(2, number + 1): 
        print(f"Task {name}: Compute factorial({i})...")
        f *= i
        await asyncio.sleep(0) 
    print(f"Task {name}: factorial({number}) = {f}") 

async def main(): 
    await asyncio.gather(
        factorial("A", 15),
        factorial("B", 7),
        factorial("C", 4),)

if __name__ == '__main__':
    await main()

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: Compute factorial(3)...
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task A: Compute factorial(4)...
Task B: Compute factorial(4)...
Task C: Compute factorial(4)...
Task A: Compute factorial(5)...
Task B: Compute factorial(5)...
Task C: factorial(4) = 24
Task A: Compute factorial(6)...
Task B: Compute factorial(6)...
Task A: Compute factorial(7)...
Task B: Compute factorial(7)...
Task A: Compute factorial(8)...
Task B: factorial(7) = 5040
Task A: Compute factorial(9)...
Task A: Compute factorial(10)...
Task A: Compute factorial(11)...
Task A: Compute factorial(12)...
Task A: Compute factorial(13)...
Task A: Compute factorial(14)...
Task A: Compute factorial(15)...
Task A: factorial(15) = 1307674368000
